### Parent-Document-Retriever Test

In [ ]:
"""
< Parent-Document-Retriever >

유사 청크(Text Chunk)를 찾은 후, 부모 문서(Parent Document)에 할당된 다른 텍스트 청크들도 참조하여
LLM에게 보냄 (결국 Parent Document를 참고하게 됨)

ex)

Document: 
        + Page 1
             - Text Chunk 1-1
             - Text Chunk 2-2
             - Text Chunk 3-3         
         + Page 2
             - Text Chunk 1-1
             - Text Chunk 2-2
             - Text Chunk 3-3         
         
'Page 1'에 있는 'Text Chunk 1-1' 이 질문과 유사하다고 판단되면 Page1의 다른 청크들도 참고히여
LLM에게 넘겨주게 됨. 이렇게 사용하면, 앞 뒤 문맥을 더 참고하여 답변할 수 있음.
(유사 문서의 부모 문서를 참고하므로, 조금 더 맥락을 담아 LLM에게 제공 가능)
(Retriever에 llm 모델이 선언되지 않고 사용됨...)


** Parent Document의 싸이즈가 너무 클 경우에는:
            - Parent Splitter & Child Splitter 2개를 나눠서 사용
            - https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/parent_document_retriever/

"""

In [1]:
# Libraries
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
# PDF 파일 불러오기 및 페이지별로 저장
loaders = [
    PyPDFLoader("./data/케이뱅크.pdf"),
    PyPDFLoader("./data/토스뱅크.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

In [6]:
# Embedding load
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {"normalize_embeddings":True}
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    encode_kwargs = encode_kwargs
)


C:\Users\jongh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Create Child Documents - text splitter - 자식 문서 
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# VectorStore 구축
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=ko_embedding
)

# Parent Documents Retriever
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter
)
retriever.add_documents(docs, ids=None)

In [9]:
sub_docs = vectorstore.similarity_search("흑자를낸 뱅크는?")

In [10]:
sub_docs

[Document(page_content='은행  (NEUTRAL ) \n케이뱅크  – 2Q 분기 흑자 전환 성공  \n \n• 케이뱅크는  2Q21 28.1% q-q의 견조한  자산 성장률을  기록한  가운데 , 39억원의  흑자\n전환에  성공. 작년 분기별  평균 263억원의  적자 시현 감안할 때, 이익 개선이  빠르\n게 이뤄진  것을 방증. \n• 1) 대출 고성장으로  이자이익이  396.4 % y-y, 71.3% q-q 증가와 함께 2) 수수료이익\n이 85억원의  순이익을  기록한  것이 실적 개선을  견읶. \n• 지난 7월 완료된  1.25조원의 유상증자는  케이뱅크  실적의  본격적읶  개선을  위한 기\n반이 될 전망. 향후 성장의  지속성을  위한 관건은  고객 기반 활용도  제고로 판단.  \nWHAT’S THE STORY  \n케이뱅크  2Q21 실적 - 적자폭  크게 감소: 케이뱅크는  2Q21 28.1% q -q의 견조한  자산 성장', metadata={'doc_id': 'dcdc4a93-a3a0-438f-a9b2-6e8d17ecd438', 'page': 0, 'source': './data/케이뱅크.pdf'}),
 Document(page_content='토스뱅크  (비상장 ) \n2Q 리뷰: 적자 폭 축소 - 흑자 전환 기간 단축이  관건 \n \n• 2Q23 당기순손실  105억원 시현. 적자 폭 축소 추세 긍정적 .  \n• 순이자이익  증가 및 CCR 하락은  긍정적이나 , 현재의  매크로  상황 및 동사 여신 포트\n폴리오  감안 시, 여전히  자산 건전성  관리가  최대 관건 중 하나.  \n• 7월 중 흑자 전환에  성공한  것으로  보도된  가운데 , 흑자 전환 성공을  통한 선순환  고\n리 창출 여부가  향후 최대 관건. \nWHAT’S THE STORY?  \n2Q23 당기순손실  105억원 시현 – 적자 폭 감소: 토스뱅크는  2Q23 105억원의  순손실을  기\n록. 순손실  규모는  3Q22 476

In [11]:
# 질문에 관련하여 가장 유사한 0번째 문서 - 답변
print("글 길이: {}\n\n".format(len(sub_docs[0].page_content)))
print(sub_docs[0].page_content)

글 길이: 455


은행  (NEUTRAL ) 
케이뱅크  – 2Q 분기 흑자 전환 성공  
 
• 케이뱅크는  2Q21 28.1% q-q의 견조한  자산 성장률을  기록한  가운데 , 39억원의  흑자
전환에  성공. 작년 분기별  평균 263억원의  적자 시현 감안할 때, 이익 개선이  빠르
게 이뤄진  것을 방증. 
• 1) 대출 고성장으로  이자이익이  396.4 % y-y, 71.3% q-q 증가와 함께 2) 수수료이익
이 85억원의  순이익을  기록한  것이 실적 개선을  견읶. 
• 지난 7월 완료된  1.25조원의 유상증자는  케이뱅크  실적의  본격적읶  개선을  위한 기
반이 될 전망. 향후 성장의  지속성을  위한 관건은  고객 기반 활용도  제고로 판단.  
WHAT’S THE STORY  
케이뱅크  2Q21 실적 - 적자폭  크게 감소: 케이뱅크는  2Q21 28.1% q -q의 견조한  자산 성장
